In [24]:
{-# LANGUAGE GeneralizedNewtypeDeriving #-}
{-# LANGUAGE TypeSynonymInstances #-}
{-# LANGUAGE FlexibleInstances #-}
{-# LANGUAGE RankNTypes #-}
{-# LANGUAGE BinaryLiterals #-}


import Data.Bits             (Bits (bit, complement, popCount, shiftR, (.&.), (.|.), testBit),
                              FiniteBits (finiteBitSize))
import Data.ByteArray.Hash   (FnvHash32 (..), fnv1Hash)
import Data.ByteString.Char8 (pack)
import Data.Char             (intToDigit)
import Data.Semigroup        ((<>))
import Data.Vector           (Vector, drop, singleton, take, (!), (//))
import Data.Word             (Word16, Word32)
import Numeric               (showIntAtBase)
import Prelude               hiding (drop, lookup, take)
import System.TimeIt         (timeIt)
import Text.Show.Pretty      (pPrint)

In [25]:
newtype Binary a = Binary a
    deriving (Enum, Ord, Real, Integral, Eq, Num, Bits, FiniteBits)

instance (FiniteBits a, Show a, Integral a) => Show (Binary a) where
    show (Binary n) = let
        str = showIntAtBase 2 intToDigit n ""
        size = finiteBitSize n
        in replicate (size - length str) '0' <> str

In [26]:
type Hash = Binary Word16

class Hashable a where
    hash :: a -> Hash

In [27]:
instance Hashable String where
    hash s = let
        FnvHash32 h = fnv1Hash (pack s)
        in Binary (fromIntegral h)

instance Hashable Int where
    hash int = Binary (fromIntegral int)

In [29]:
class Mapping mapping where
    empty :: forall k v. mapping k v
    lookup :: forall k v. (Hashable k) => k -> mapping k v -> Maybe v
    insert :: forall k v. (Hashable k) => k -> v -> mapping k v -> mapping k v

In [28]:
data HashBinaryMappedTreeData key value = HashBinaryMappedTreeData Hash key value deriving (Eq, Show)

data HashBinaryMappedTree key value
    = HashBinaryMappedTreeLeaf
    | HashBinaryMappedTreeNode
        (HashBinaryMappedTree key value)
        (Maybe (HashBinaryMappedTreeData key value))
        (HashBinaryMappedTree key value)
    deriving (Eq, Show)

In [30]:
emptyHashBinaryMappedTree = HashBinaryMappedTreeNode
    HashBinaryMappedTreeLeaf
    Nothing
    HashBinaryMappedTreeLeaf

In [34]:
insertHashBinaryMappedTree :: (Hashable key) => key -> value -> HashBinaryMappedTree key value -> HashBinaryMappedTree key value
insertHashBinaryMappedTree key = insertHashBinaryMappedTreeHelper 0 (hash key) key

insertHashBinaryMappedTreeHelper :: Int -> Hash -> key -> value -> HashBinaryMappedTree key value -> HashBinaryMappedTree key value
insertHashBinaryMappedTreeHelper depth hash key value tree
    -- If the remainder of the hash is zero, we update the value at the current node
    | hash `shiftR` depth == 0 = case tree of
        HashBinaryMappedTreeLeaf -> HashBinaryMappedTreeNode
            HashBinaryMappedTreeLeaf
            (Just (HashBinaryMappedTreeData hash key value))
            HashBinaryMappedTreeLeaf
        HashBinaryMappedTreeNode l _ r -> HashBinaryMappedTreeNode
            l
            (Just (HashBinaryMappedTreeData hash key value))
            r
    -- Otherwise we go left if the current bit is 0 or right if the current bit is 1 and recursively insert
    | otherwise = let
        goRight = testBit hash depth
        depth' = depth + 1
        in case tree of
            HashBinaryMappedTreeLeaf -> if goRight
                then HashBinaryMappedTreeNode
                    HashBinaryMappedTreeLeaf
                    Nothing
                    (insertHashBinaryMappedTreeHelper depth' hash key value HashBinaryMappedTreeLeaf)
                else HashBinaryMappedTreeNode
                    (insertHashBinaryMappedTreeHelper depth' hash key value HashBinaryMappedTreeLeaf)
                    Nothing
                    HashBinaryMappedTreeLeaf
            HashBinaryMappedTreeNode l d r -> if goRight
                then HashBinaryMappedTreeNode
                    l
                    d
                    (insertHashBinaryMappedTreeHelper depth' hash key value r)
                else HashBinaryMappedTreeNode
                    (insertHashBinaryMappedTreeHelper depth' hash key value l)
                    d
                    r

In [36]:
lookupHashBinaryMappedTree :: (Hashable key) => key -> HashBinaryMappedTree key value -> Maybe value
lookupHashBinaryMappedTree key = lookupHashBinaryMappedTreeHelper 0 (hash key) key

lookupHashBinaryMappedTreeHelper :: Int -> Hash -> key -> HashBinaryMappedTree key value -> Maybe value
lookupHashBinaryMappedTreeHelper depth hash key tree
    -- If the remainder of the hash is zero, we check the value at the current node
    | hash `shiftR` depth == 0 = case tree of
        HashBinaryMappedTreeLeaf -> Nothing
        HashBinaryMappedTreeNode _ d _ -> case d of
            Nothing -> Nothing
            Just (HashBinaryMappedTreeData dataHash _dataKey dataValue) -> if hash == dataHash
                then Just dataValue
                else Nothing
    -- Otherwise we go left if the current bit is 0 or right if the current bit is 1 and recursively lookup
    | otherwise = let
        goRight = testBit hash depth
        depth' = depth + 1
        in case tree of
            HashBinaryMappedTreeLeaf -> Nothing
            HashBinaryMappedTreeNode l _ r -> if goRight
                then lookupHashBinaryMappedTreeHelper depth' hash key r
                else lookupHashBinaryMappedTreeHelper depth' hash key l